<a href="https://colab.research.google.com/github/husseinahmad7/LLmExercise/blob/main/Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb sentence-transformers gradio flask pypdf transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.1 MB/s eta 0:00:00


In [32]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load PDF
loader = PyPDFLoader("FR_FAQs.pdf")
documents = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


In [33]:
from sentence_transformers import SentenceTransformer
import chromadb

# Load sentence transformer model for embeddings
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create Chroma client and collection
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="faqs_collection")

collection.add(
    documents=[doc.page_content for doc in docs],
    metadatas=[{"source": doc.metadata["source"]} for doc in docs],
    ids=[str(i) for i in range(len(docs))]
)
# Embed documents and add to collection
# for i, doc in enumerate(docs):
#     embedding = embedding_model.encode(doc.page_content).tolist()  # Convert ndarray to list
#     collection.add(documents=[doc.page_content], embeddings=[embedding], ids=[str(i)])


In [5]:
# !git clone https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2


In [6]:
# from huggingface_hub import hf_hub_download, snapshot_download

# snapshot_download('m-a-p/neo_7b')
# hf_hub_download(repo_id="mistralai/Mistral-7B-Instruct-v0.2", filename="config.json", cache_dir="./mistralai/Mistral-7B-Instruct-v0.2")

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"
# model_name = 'Writer/camel-5b-hf'
# model_path = '/root/.cache/huggingface/hub/models--m-a-p--neo_7b/snapshots/7abd89ce6d10077dad209afe27a20767b273fbfa'
tokenizer = AutoTokenizer.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [8]:
# !pip install -Uqqq


# !pip -qqq install bitsandbytes accelerate

In [ ]:
# import accelerate

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_name,
    torch_dtype='auto').eval()



model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [12]:

class HuggingFaceLLMWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt").to('cpu')
        outputs = self.model.generate(**inputs, max_length=150)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

llm = HuggingFaceLLMWrapper(model=model.to('cpu'), tokenizer=tokenizer)

In [34]:
from langchain.vectorstores import Chroma


# Define a function to embed queries
def embed_query(query):
    return embedding_model.encode([query])[0].tolist()

# Initialize Chroma vector store
vector_db = Chroma(client=chroma_client, collection_name="faqs_collection", embedding_function=embed_query)

# Create a retriever from the vector store
retriever = vector_db.as_retriever()

In [39]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Define the prompt template for question-answering
qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Context: {context}\n\nQuestion: {question}\n\nAnswer:"
)

# Initialize HuggingFace LLM wrapper
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=llm_pipeline)
# llm = HuggingFacePipeline(pipeline=model)

# Load the QA chain
# question_answer_chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=qa_prompt)

# # Create the retrieval chain
# qa_chain = RetrievalQA(retriever=retriever, combine_documents_chain=question_answer_chain)


In [40]:
# Create the question-answering chain
from langchain.chains.combine_documents import create_stuff_documents_chain
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)


In [41]:
# Create the retrieval chain
from langchain.chains import create_retrieval_chain
qa_chain = create_retrieval_chain(retriever, question_answer_chain)



In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain

# # Define the prompt template for question-answering
# qa_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="Context: {context}\n\nQuestion: {question}\n\nAnswer:"
# )

# # Create the question-answering chain
# question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

# # Create the retrieval chain
# qa_chain = create_retrieval_chain(retriever=retriever, question_answer_chain=question_answer_chain)


In [42]:
import gradio as gr

# Function to answer questions using the QA chain
def answer_question(question):
    result = qa_chain({"query": question})
    return result["result"]

# Create Gradio interface
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="FAQ Answering System",
    description="Ask questions about the FAQ and get answers based on the content of the PDF."
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a956c186bfe9a7bf68.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def answer_question(question):
    retrieved_docs = retriever.get_relevant_documents(question)
    result = qa_chain.run(input_documents=retrieved_docs, question=question)
    return result

# Create Gradio interface
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="FAQ Answering System",
    description="Ask questions about the FAQ and get answers based on the content of the PDF."
)

# Launch the interface
iface.launch()
